# Import Historical Price Data

In [1]:
import os
os.chdir(os.environ['PWD'])

In [2]:
import pandas as pd
import requests
from tqdm import tqdm
from openpyxl import load_workbook
from app.conf import settings as sts

In [3]:
raw_data_urls = [
    "series-historicas-precios-mayoristas-06-12-22.xlsx",
    "series-historicas-precios-mayoristas-2021.xlsx",
    "series-historicas-precios-mayoristas-2020.xlsx",
    "series-historicas-precios-mayoristas-2019.xlsx",
    "series-historicas-precios-mayoristas-2018.xlsx",
    "series-historicas-precios-mayoristas.xlsx"
]

In [4]:
def load_dataset(filename, local_path = sts.LOCAL_DATA_PATH, url_path = sts.REMOTE_URL_PATH):
    
    file_path = local_path + filename
    
    if filename not in os.listdir(local_path):
    
        url = url_path + filename
        print(url)
        resp = requests.get(url)
        assert resp.status_code == 200, resp.status_code

        file = open(file_path, 'wb')
        file.write(resp.content)
        file.close()
    
    workbook = load_workbook(file_path, data_only = True)
    sheetnames = workbook.sheetnames
    
    _dataframe_list = []
    
    for sheet in sheetnames[1:]:
        dataframe = pd.DataFrame(workbook[sheet].values)
        _valid_index = pd.to_datetime(dataframe[0], errors='coerce').dropna(axis = 0).index
        dataframe = dataframe.filter(_valid_index, axis = 0).dropna(axis = 1)
        _dataframe_list.append(dataframe)
    
    concat_dataframe = pd.concat(_dataframe_list, ignore_index=True)
    
    return concat_dataframe

In [5]:
_final_dataframe_list = []
for url in tqdm(raw_data_urls):
    dataframe = load_dataset(url)
    _final_dataframe_list.append(dataframe)
final_dataframe = pd.concat(_final_dataframe_list, ignore_index=True)
final_dataframe.columns = ['date', 'group', 'product', 'market', 'mean_price']

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:21<00:00,  3.56s/it]


In [6]:
final_dataframe['date'] = pd.to_datetime(final_dataframe['date'])

In [7]:
final_dataframe.sort_values('date', inplace=True, ignore_index=True)

In [8]:
final_dataframe.to_csv(sts.LOCAL_DATA_PATH+sts.FINAL_PRICE_DATASET, index=False)